<a href="https://colab.research.google.com/github/easyhardhoon/deep_learning_frameworks/blob/main/Pytorch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pytorch3에 대한 복습입니다

✈데이터 불러오기 방법 3가지

    어떤 방법이든 

    데이터 전처리 + DataLoader배치조정 의 형태는 동일합니다

In [1]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader, Dataset
import numpy as np

**1. 파이토치 제공 데이터 사용**

In [2]:
transf = tr.Compose([tr.Resize(8), tr.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train= True, download = True, transform=transf)
trainloader = DataLoader(trainset,batch_size=50,shuffle=True,num_workers=2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


**2. 같은 클래스 별 폴더 이미지 데이터 이용**

In [ ]:
#./class/tiger, ./class/lion 와 같이 class 하위 폴더에 각각의 데이터 폴더가 있을때
transf = tr.Compose([tr.Resize(16), tr.ToTensor()])
trainset = torchvision.datasets.ImageFolder(root='./class', transform=transf)
trainloader = DataLoader(trainset,batch_size=1,shuffle=False,num_workers=2)

**3.1 개인 데이터 사용(transform 도움없이)**

In [4]:
train_images = np.random.randint(256,size=(20,32,32,3))
train_labels = np.random.randint(2,size=(20,1))
#import preprocessing
#train_images, train_labels = preprocessing(train_images,train_labels) 
# 위 코드는 "일련의 전처리 과정"을 라이브러리로 만들어서 import 한것입니다. 
# 결국 수동 전처리라는 개념에서 동일하다고 생각됩니다.
class TensorData(Dataset): #수동 전처리를 위한 클래스입니다
  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.x_data = self.x_data.permute(0,3,1,2) 
    self.y_data = torch.LongTensor(y_data) 
    self.len = self.y_data.shape[0] 
  def __getitem__(self,index):
      return self.x_data[index], self.y_data[index]
  def __len__(self):
    return self.len
train_data = TensorData(train_images,train_labels)
train_loader = DataLoader(train_data, batch_size=10,shuffle=True)

**3.2 개인 데이터 사용(transform 도움받고)**

In [5]:
train_images = np.random.randint(256,size=(20,32,32,3))
train_labels = np.random.randint(2,size=(20,1))
class MyDataset(Dataset): 
  def __init__(self,x_data,y_data,transform=None):
    self.x_data = x_data
    self.y_data = y_data 
    self.transform = transform
    self.len = len(y_data)
  def __getitem__(self,index):
      sample = self.x_data[index], self.y_data[index]
      if self.transform: #transform이 활성화되면 transform의 도움을 받는다
        sample= self.transform(sample) 
      return sample
  def __len__(self):
    return self.len
class ToTensor: #tr.ToTensor를 사용할 수 없기 때문에 수동설정.
  #이유는 데이터 타입이 현재 PIL image가 아닌 numpy이기 때문입니다.
  def __call__(self,sample):
    inputs, labels = sample
    inputs = torch.FloatTensor(inputs)
    inputs = inputs.permute(2,0,1)
    return inputs, torch.LongTensor(labels)
class LinearTensor:
  def __init__(self,slope=1,bias=0):
    self.slope = slope
    self.bias = bias
  def __call__(self,sample):
    inputs,labels=sample
    inputs= self.slope*inputs+self.bias
    return inputs,labels
trans = tr.Compose([ToTensor(), LinearTensor(2,5)]) 
ds1 = MyDataset(train_images,train_labels,transform=trans) 
train_loader1 = DataLoader(ds1, batch_size=10,shuffle=True) 

**3.3 개인 데이터 사용(transform 도움받고 & 전처리함수 수동생성 x)**

In [6]:
train_images = np.random.randint(256,size=(20,32,32,3))
train_labels = np.random.randint(2,size=(20,1))
class MyDataset(Dataset): 
  def __init__(self,x_data,y_data,transform=None):
    self.x_data = x_data
    self.y_data = y_data 
    self.transform = transform
    self.len = len(y_data)
  def __getitem__(self,index):
      sample = self.x_data[index], self.y_data[index]
      if self.transform: 
        sample= self.transform(sample) 
      return sample
  def __len__(self):
    return self.len

class MyTransform:
  def __call__(self,sample):
    inputs, labels = sample
    inputs = torch.FloatTensor(inputs)
    inputs = inputs.permute(2,0,1)
    labels = torch.FloatTensor(labels)
    transf = tr.Compose([tr.ToPILImage(), tr.Resize(128), tr.ToTensor()]) 
    final_output = transf(inputs)
    return final_output, labels
ds2 = MyDataset(train_images, train_labels, transform=MyTransform())
train_loader2 = DataLoader(ds2, batch_size=10,shuffle=True)

#인공신경망 구현(이미지. CNN위주)

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms #기본적으로 전처리를 위한 라이브러리입니다
from torch.utils.data import DataLoader #기본적으로 배치 형태를 위한 라이브러리입니다

**Load data set(데이터 불러오기의 "파이토치 기본 제공 데이터"에 해당된다)**

In [8]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]) #tr을 그냥 transforms라고 쓴 것 뿐. 원래 transforms의 약자가 tr이었음
trainset = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=8,shuffle=True,num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset,batch_size=8,shuffle=False,num_workers=2)
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

Files already downloaded and verified
Files already downloaded and verified


**Build a model**

In [9]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module): #nn.Module을 상속받는다.
  def __init__(self): #연산 세팅
    super(Net,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5) #합성곱
    self.pool = nn.MaxPool2d(2,2) #Max풀링
    self.conv2 = nn.Conv2d(6,16,5) #합성곱
    self.fc1 = nn.Linear(16*5*5,120) #완전연결계층
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)
  def forward(self,x): #연산 순서 세팅. 순전파
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1,16*5*5) #사이즈 변형. 완전연결계층에 연결하기 위해선 데이터를 1자로 펴야한다
    x = F.relu(self.fc1(x)) # 단순 fc(x)는 행렬곱연산이라 생각하면 됩니다.
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
net = Net()


#복습

Fully connected layer : 완전연결 계층

한층의 모든 뉴런이 다음층의 모든 뉴런과 연결된 상태.

✈2차원의 배열 형태 이미지를 1차원의 평탄화 작업을 통해 이미지를 분류하는데 사용

✈활성화 함수 뉴런  활성화

✈분류기 함수(softmax)로 분류

즉, 완전연결 계층이란

**Flatten(평탄화) + 활성화함수(Relu...) + 활성화함수(softmax...)**



In [10]:
net

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

**Implement the model with training set**

In [11]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) #net안의 parameter들을 업데이트 해주겠다는 의미
#학습률 0.001, 근데 SGD 방식인데 momentum을 왜 쓰지..?

for epoch in range(1): #epoch. 데이터 전체를 몇번 보느냐
  running_loss = 0.0
  for i,data in enumerate(trainloader,0): #당연히 앞에서 전처리에 배치처리까지 마친 데이터 셋이 들어가야지
    inputs,labels=data
    
    optimizer.zero_grad() #파라미터의 grad값(미분값)을 0으로  초기화. 

    outputs= net(inputs) #산출값
    loss = criterion(outputs,labels) #손실함수를 계산합니다. 당연히 산출값과 정답 레이블의 오차를 계산합니다 
    #주의 : batch형태로 들어와도 loss 결과값은 무조건 스칼라값, 즉 하나의 상수 값입니다.
    loss.backward() #역전파를 돌립니다. gradient(미분값)을 구합니다. loss 기준!!!!!
    optimizer.step() #가중치 매개변수 값을 SGD 방식으로 업데이트 합니다.

    running_loss += loss.item() #누적 loss 값을 더해줍니다. 
    if i % 2000 == 1999:
      print('[%d, %5d] loss : %.3f' % (epoch+1,i+1,running_loss/2000))
      running_loss = 0.0
print("Finished Training")

[1,  2000] loss : 2.145
[1,  4000] loss : 1.750
[1,  6000] loss : 1.564
Finished Training


**위 코드는 딥러닝 실행의 표준 코드입니다**

**아래는 딥러닝_학습의 전체 흐름입니다**

1. 데이터 가져오기, 전처리 하기, 배치화하기

2. criterion(loss구하는법), opimizer(가중치매개변수 업데이트하는법) 설정, 초기화

3. 만들어놓은 Net에 배치화된 데이터 넣기. ouput 산출

4. criterion을 통해 ouput과 정답 label 비교. **단일 스칼라값 형태의 loss 산출**

5. loss 기준의 backward 계산(미분값 계산)

6. 미분 계산값(loss함수에 대한 가중치 매개변수의 기울기)을 토대로 optimizer를 통해 가중치 매개변수 업데이트

7. 출력을 위해 loss 값 누적 & 결과값 산출

8. 반복( 입력 데이터의 각 배치가 모두 끝날때까지)

9. 반복을 반복(epoch)

⏰batch : 데이터를 batch로 묶는다.  

ex) 10개의 데이터를 2 크기의 batch로 묶는다. --> 전체 데이터는 5개의 batch로 구성된다

⏰epoch : 전체 데이터를 몇번 반복해서 보느냐 

ex)5개의 batch로 이루어진 데이터를 한바퀴 본다 --> 1 epoch


ex)5개의 batch로 이루어진 데이터를 10바퀴 반복해서 본다 --> 10 epoch

**Save the trained model**



In [13]:
#앞서 힘들게 계산한  loss값들을 다시 또 안돌리려면 저장이 필요하다
PATH = './cifar_net.pth'
torch.save(net.state_dict(),PATH)
#참고로 model save는 적재적소에 가능함. 위의 학습코드에서 3epoch일때 가장 적합한 loss가 나올수 있음
#그때 조건문을 걸어 바로 save하게 코드구현이 가능함

**Load the pre-trained model**

In [15]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

**이제 테스트해보자!!**

In [17]:
correct = 0
total = 0
with torch.no_grad(): # gradient 업데이트 기능을 안하면서 하겠다는 의미
  for data in testloader:
    images,labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1) #torch.max(outputs.data,1)을 하게 되면 (,2)과 같은 1차원 튜플값이 나온다.
    #참고로 torch.max는 argmax와 같은 역할임. batch내의 각 데이터마다 최댓값(ex. 0.8 원핫인코딩.)인 인덱스를 리턴. 
    #그래서 ,을 무시하기 위해 언더스코어 사용
    total +=  labels.size(0)
    correct += (predicted == labels).sum().item() #같으면 맞는거다. 이 맞는거의 총합 sum. 
print('Accuracy of the network on the 10000 test images : %d %%' %(100*correct/total))

Accuracy of the network on the 10000 test images : 44 %


⏰python에서 _(언더스코어)의 의미

1. 마지막으로 실행된 결과값 저장 (in 인터프리터)

    ex) 10을 쓰고 enter 했다면 다음 쉘에서 _를 치고 enter하면 10이 나옴

2. **값을 무시하고 싶은 경우**

    ex) x, _, y = (1,2,3) ---> #x=1, y=3


3. 특별한 의미의 네이밍을 하는 경우

    private 클래스/함수/변수/매서드를 선언할때

    ex) class _Base: #private 클래스

    def __init__  

    def _double #private 매서드

이미지처리는 사실 위와 같은 CPU연산로 하면 엄청나게 느리고 오래걸림

GPU연산으로 하면 훨씬 효율적이다..! 이는 다음시간에